In [2]:
import torch
import numpy as np
import seaborn as sns

import sys
sys.path.append('..')
import datafree

In [4]:
distributed = False
gpu = 4
batch_size = 256
workers = 8
num_classes = 10
def prepare_model(model):
    if not torch.cuda.is_available():
        print('using CPU, this will be slow')
        return model
    elif distributed:
        # For multiprocessing distributed, DistributedDataParallel constructor
        # should always set the single device scope, otherwise,
        # DistributedDataParallel will use all available devices.
        if args.gpu is not None:
            torch.cuda.set_device(gpu)
            model.cuda(gpu)
            # When using a single GPU per process and per
            # DistributedDataParallel, we need to divide the batch size
            # ourselves based on the total number of GPUs we have
            batch_size = int(batch_size / ngpus_per_node)
            workers = int((workers + ngpus_per_node - 1) / ngpus_per_node)
            model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[gpu])
            return model
        else:
            model.cuda()
            model = torch.nn.parallel.DistributedDataParallel(model)
            return model
    elif gpu is not None:
        torch.cuda.set_device(gpu)
        model = model.cuda(gpu)
        return model
    else:
        # DataParallel will divide and allocate batch_size to all available GPUs
        model = torch.nn.DataParallel(model).cuda()
        return model

In [5]:
from torchvision.datasets import CIFAR10
import registry
student = registry.get_model('resnet18', num_classes=num_classes)
teacher = registry.get_model('resnet34', num_classes=num_classes, pretrained=True).eval()
normalizer = datafree.utils.Normalizer(**registry.NORMALIZE_DICT['cifar10'])
student = prepare_model(student)
teacher = prepare_model(teacher)

In [7]:
import numpy as np
num_classes, ori_dataset, val_dataset = registry.get_dataset(name='cifar10', data_root='/data/lijingru/cifar10/')
random_idx = np.random.randint(0, len(ori_dataset), (256,))

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
x_feats = []
for idx in random_idx:
    x, y = ori_dataset[idx]
    x = x.to(gpu)
    output, x_feat = teacher(x)
    x_feats.append(x_feat)